In [1]:
from keras.models import load_model
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import cv2
import scipy.io
#ROI 
x = [200, 600]
y = [40, 650]
#x = [0, 576]
#y = [0, 768]
original_sizes = []
def load_images_from_folder(path):
  image = []
  mask = []
  i = 0
  files= os.listdir(path) #得到資料夾下的所有檔名稱
  #orginal size(576, 768, 3)
  for root, dirs, files in os.walk(path):
    for file in files:
      fullpath = os.path.join(root, file)
      if file.find(".png") != -1:
        img = np.array(cv2.imread(fullpath, cv2.IMREAD_GRAYSCALE))#, cv2.IMREAD_GRAYSCALE 
        original_sizes.append(img.shape)
        img = img[x[0]:x[1], y[0]:y[1]]
        image.append(img)
  image = np.array(image)
  return image

In [4]:
from scipy.signal import argrelextrema
from scipy.ndimage.filters import gaussian_filter

# input: img
# return: cropImgUp(沿血管中心水平裁切後的img上半部+垂直翻轉 aka翻轉後的血管上壁)
#         cropImgDown(沿血管中心水平裁切後的img下半部 aka血管下壁)
#         cropLineUp(上半部img的上界)
#         axis(血管的軸 aka第幾個row)
#         cropLineDown(下半部img的下界)

def get_cropImg_axis(img):

    # 把img每個row所有pixel的intensity value相加存進row_sum
    row_size = img.shape[0]
    row_sum = np.zeros(row_size)
    for i in range(row_size):
        row_sum[i] = np.sum(img[i,:])
    row_sum = gaussian_filter(row_sum, sigma=7)

    # 從row_sum中找出所有的local maximum
    local_max = argrelextrema(row_sum, np.greater)
    max_value =  row_sum[local_max]   
    max_value = np.sort(max_value)

    # 如果local maximum的數量>=3，就取出值前三大的local maximum，假設為血管壁位置
    if len(max_value)>=3:
        max1 = max_value[-1]
        max2 = max_value[-2]
        max3 = max_value[-3]
        for i in range(row_size):
            if row_sum[i] == max1:
                wall1 = i
            elif row_sum[i] == max2:
                wall2 = i
            elif row_sum[i] == max3:
                wall3 = i
        # 上下血管壁位置取中間即為血管軸
        # 算出三組可能為血管軸的位置，找出intensity value最小者
        # 並記錄下此狀況下，血管壁的位置
        axis1 = (wall1 + wall2)//2
        axis2 = (wall2 + wall3)//2
        axis3 = (wall1 + wall3)//2
        axis = axis1
        wallUp = wall1
        wallDown = wall2
        if row_sum[axis]>row_sum[axis2]:
            axis = axis2
            wallUp = wall2
            wallDown = wall3
        if row_sum[axis]>row_sum[axis3]:
            axis = axis3
            wallUp = wall1
            wallDown = wall3

    #如果local maximum的數量>=，兩血管壁位置取中間即為血管軸
    elif len(max_value)>=2:
        max1 = max_value[-1]
        max2 = max_value[-2]
        for i in range(row_size):
            if row_sum[i] == max1:
                wall1 = i
            elif row_sum[i] == max2:
                wall2 = i
        axis = (wall1 + wall2)//2
        wallUp = wall1
        wallDown = wall2

    else:
        print('error')
        return

    if wallUp>wallDown:
        temp = wallDown
        wallDown = wallUp
        wallUp = temp
    axis = int(axis)#
    # 透過血管壁與軸的位置做裁切，裁切邊界必須在img內
    cropLineUp = wallUp-(axis-wallUp)   
    if cropLineUp<0:
        cropLineUp = 0
    cropLineDown = wallDown+(wallDown-axis) 
    if cropLineDown>img.shape[0]:
        cropLineDown = img.shape[0]

    # 回傳裁切後圖片以及裁切軸

    cropImgDown = img[axis:cropLineDown,:]
    cropImgUp = img[cropLineUp:axis,:]
    cropImgUp = cv2.flip(cropImgUp, 0)

    return cropImgUp, cropImgDown, cropLineUp, axis, cropLineDown

In [5]:
#sobel和prewitt filter 
def imgsToSobel(img):
  sobely = cv2.Sobel(img,cv2.CV_8U,0,1,ksize=3)
  return np.array(sobely)
def imgsToPrewitt(img):
  #kernelx = np.array([[1, 1, 1], [0, 0, 0], [-1, -1, -1]], dtype=int)
  kernelx = np.array([[-1.,-1.,-1.],[0.,0.,0.],[1.,1.,1.]], dtype=np.float64)
  kernely = np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]], dtype=np.float64)
  x = cv2.filter2D(img.astype(np.float64), -1, kernelx,borderType = cv2.BORDER_CONSTANT)
  y = cv2.filter2D(img.astype(np.float64), -1, kernely,borderType= cv2.BORDER_CONSTANT)
  x = np.clip(x, 0, 255).astype(np.uint8)
  y = np.clip(y, 0, 255).astype(np.uint8)

  Prewitt = y-x
  return 255-Prewitt


In [6]:
#這邊做兩件事，先經過兩個filter，再resize 成model的input size
def produce_preprocessed_img(imgs):
  #input (79,512,512)
  img = []
  for image in imgs:
    sobel = imgsToSobel(image)
    prewitt = imgsToPrewitt(image)
    img.append([image, sobel, prewitt])
  return np.array(img)
  #return (79,3,512,512,3)

In [7]:
#用model標出上下血管壁內膜
def do_seg(img, sobel_img, prewitt_img):
    testimg = img
    input1 = sobel_img.reshape(-1,128,512,1).astype("float32")/255
    input2 = prewitt_img.reshape(-1,128,512,1).astype("float32")/255
    seg = model.predict([input1, input2])
    seg = seg.reshape(128,512)
    seg= (seg*255).astype(np.uint8)
    
    #plt.hist(seg.ravel(), bins=20)
    #plt.show()
    
    #plt.imshow(seg,cmap='gray')
    #plt.show()
    ret, seg = cv2.threshold(seg, 40, 255, cv2.THRESH_BINARY)
    #threshhold 可以設5
    #這邊要找一下有沒有更好的方法
    return seg
    

In [8]:
model_path = "drive/MyDrive/大三專題/100-IMT-ImagesCY/model0416_sobel_prewitt(faster).h5"
model_path = "drive/MyDrive/大三專題/100-IMT-ImagesCY/model0811_sobel_prewitt.h5"
model = load_model(model_path)


In [9]:
#np.random.seed(20010319)
#np.random.shuffle(img)

In [10]:
# 去除假影(廠商機器產生的)
def eli_noise(img, dark, percentile):
    remove_dark = img[img>dark]
    AIV = np.percentile(remove_dark, percentile)
    multiply = 255/(255-AIV)
    img = np.where(img >= AIV, (img-AIV)*multiply, 0)
    return img 


In [11]:
#把上下的血管壁切出來
def get_crop(imgs):
  topimgs = []
  botimgs = []
  cropLineUp = []
  cropLineDown = []
  bot_cropped_size = []
  top_cropped_size = []
  for img in imgs:
    Resize = (512,128)
    #return cropImgUp, cropImgDown, cropLineUp, axis, cropLineDown
    top, bot, LineUp, axis, LineDown = get_cropImg_axis(img)
    top_cropped_size.append(top.shape)
    bot_cropped_size.append(bot.shape)

    top = cv2.bilateralFilter(top,9,50,50)
    bot = cv2.bilateralFilter(bot,9,50,50)
    top = eli_noise(top, 20, 60)
    bot = eli_noise(bot, 20, 15)
    
    top = cv2.resize(top, Resize, interpolation = cv2.INTER_AREA)
    bot = cv2.resize(bot, Resize, interpolation = cv2.INTER_AREA)

    
    topimgs.append(top)
    botimgs.append(bot)
    cropLineUp.append(LineUp)
    cropLineDown.append(LineDown)


  topimgs = np.array(topimgs)
  botimgs = np.array(botimgs)
  return topimgs, botimgs, top_cropped_size, bot_cropped_size, cropLineUp, cropLineDown

In [12]:
def predict(imgs):
  #Reconstruct (paste predicted top intima masks back to bot-masks)
  topimgs, botimgs, top_cropped_size, bot_cropped_size, cropLineUp, cropLineDown = get_crop(imgs)
  topimgs = produce_preprocessed_img(topimgs)#(79,2,512,512)
  botimgs = produce_preprocessed_img(botimgs)#(79,2,512,512)
  results = []
  for i in range(len(topimgs[:,0])):
    raw = topimgs[:,0][i]
    sobel_img = topimgs[:,1][i]

    top_mask = do_seg(topimgs[:,0][i], topimgs[:,1][i], topimgs[:,2][i])
    top_mask = cv2.flip(top_mask, 0)
    bot_mask = do_seg(botimgs[:,0][i], botimgs[:,1][i], botimgs[:,2][i])

    top_mask = cv2.resize(top_mask, (top_cropped_size[i][1], top_cropped_size[i][0]), interpolation = cv2.INTER_NEAREST)
    bot_mask = cv2.resize(bot_mask, (bot_cropped_size[i][1], bot_cropped_size[i][0]), interpolation = cv2.INTER_NEAREST)
    combined = np.append(np.array(top_mask), np.array(bot_mask), axis = 0)
    #predictions
    
    #把上下貼起來
    topzero = np.zeros((x[0] + cropLineUp[i], top_cropped_size[i][1]))
    combined = np.append(topzero, combined, axis = 0)
    botzero = np.zeros(( original_sizes[i][0] - (x[0] + cropLineDown[i]), top_cropped_size[i][1]))
    combined = np.append(combined, botzero, axis = 0)#currently column is fully padded
    leftzero = np.zeros((combined.shape[0],y[0]))
    combined = np.append(leftzero, combined, axis = 1)
    rightzero = np.zeros((combined.shape[0],original_sizes[i][1]-combined.shape[1]))
    combined = np.append(combined, rightzero, axis = 1)

    results.append(combined)
  results = np.array(results)
  return results, topimgs, botimgs 

In [13]:
def show_results(imgs, results, topimgs, botimgs):
  for i in range(len(imgs)):
    alpha =0.5
    plt.imshow(imgs[i],cmap='gray')
    plt.show()
    plt.imshow(topimgs[i][0],cmap='gray')
    plt.show()
    plt.imshow(botimgs[i][0],cmap='gray')
    plt.show()
    overlap = cv2.addWeighted(imgs[i], alpha, results[i][x[0]:x[1],y[0]:y[1]].astype(np.uint8), 1-alpha,0)
    plt.imshow(overlap,cmap='gray')
    plt.show()
    print("-------------" + str(i))

In [14]:
#當時廠商給的資料集比較連續，這邊是方便看結果而以
def shuff(imgs):
  #input (79,512,512,3)
  shuff_const = 20010319
  indices = np.arange(imgs.shape[0])
  np.random.seed(shuff_const)
  np.random.shuffle(indices)
  imgs = imgs[indices]
  return imgs

In [15]:
#-------------------------------------------------------------


In [16]:
dir_train = "drive/MyDrive/大三專題/capture_image/clear_img"
imgs = load_images_from_folder(dir_train)
#imgs = shuff(imgs)
#imgs = imgs[:15]
results, topimgs, botimgs = predict(imgs)
show_results(imgs, results, topimgs, botimgs)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
dir_train = "drive/MyDrive/大三專題/capture_image/female_Old"#training
imgs = load_images_from_folder(dir_train)
#imgs = shuff(imgs)
#imgs = imgs[:15]
results, topimgs, botimgs = predict(imgs)
show_results(imgs, results, topimgs, botimgs)

In [ ]:
dir_train = "drive/MyDrive/大三專題/capture_image/female_Young"#training
imgs = load_images_from_folder(dir_train)
#imgs = shuff(imgs)
#imgs = imgs[:15]
results, topimgs, botimgs = predict(imgs)
show_results(imgs, results, topimgs, botimgs)

In [ ]:
dir_train = "drive/MyDrive/大三專題/capture_image/Liang"#training
imgs = load_images_from_folder(dir_train)
#imgs = shuff(imgs)
#imgs = imgs[:15]
results, topimgs, botimgs = predict(imgs)
show_results(imgs, results, topimgs, botimgs)